<h2> Refine2DVertex Algorithm Description </h2>

This example uses LArCV file contents.
So you will see some LArCV related data loading in this example but I try to clearly note where applicable so you can focus on algorithm part.

<h3> Module Imports </h3>
First let's just import modules.

In [ ]:
import ROOT, sys, os
from ROOT import std
#from larlite import larlite # LArLite 
from larcv import larcv # This is LArCV module, you don't have to worry about it
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.path as path
colormap=['blue','red','magenta','green','orange','yellow','pink']
nc=len(colormap)


OK now we import Geo2D package which is a dependency for Refine2DVertex module.

In [ ]:
from ROOT import geo2d,cv
from ROOT.cv import Point_ as Vector
DTYPE='float'

If you get an import error here you are probably missing Geo2D package.<br>
You can find it here: https://www.github.com/LArbys/Geo2D <br>
git pull then follow a usual procedure of building it.

Successful import? great! 
One thing to note: as you might know Geo2D is type-templated heavily.
DTYPE defined above is used to specialize Geo2D types used in this example to specialize w/ 4byte single floating point.
You can change it to a different type if you want to (but be responsible for type compatibilities here and there)

<h3> LArCV Data Process </h3>
Let's read in some LArCV data file to get an example image to work with.

In [ ]:
# Create IO
iom=larcv.IOManager(larcv.IOManager.kBOTH)        
iom.reset()
iom.set_verbosity(0) # to see some verbose output on the terminal
iom.add_in_file("mcc7_bnb_detsim_to_larcv_hires_v00_p00_out_0000.root")
iom.set_out_file("/tmp/trash.root")

# Initialize IO
iom.initialize()
# Read one entry
iom.read_entry(10)

OK, we got IO ready. Next prepare LArCV interface for LArOpenCV (called LArbysImage)

In [ ]:
larbysimg=larcv.LArbysImage()
cfg=larcv.CreatePSetFromFile("unit.fcl","LArbysImage")
larbysimg.configure(cfg)
larbysimg.initialize()

OK, the interface module is ready.
Let's process the event that we have read-in.
All we need to do is to call an event execution function of the interface module.
Again, this is LArCV jargon so don't worry too much.

In [ ]:
larbysimg.process(iom)

OK this processed the Refine2DVertex algorithm for this event. 
Let's take a look at an output.

<h3> Refine2DVertex </h3>

Let's understand what this algorithm does.
To see this, first take a look at raw images that are used to process.
This can be obtained from ImageClusterManager::InputImages function.
In the code below we show the raw images + store them in img_v list type variable.
Image shown is artificially thresholded to pixel value > 10 and binalized to 0/1.

In [ ]:
pygeo   = geo2d.PyDraw()

mgr=larbysimg.Manager()
img_v = []

xranges=((400,700),(400,700),(450,700))
yranges=((900,1100),(750,900),(1350,1600))

plane=0
for mat in mgr.InputImages():
    #img_v.append(pygeo.image(mat).transpose())
    print 'Plane',plane
    img_v.append(pygeo.image(mat))
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    fig,ax=plt.subplots(figsize=(12,12))
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    ax.set_ylim(yranges[plane][0],yranges[plane][1])
    ax.set_xlim(xranges[plane][0],xranges[plane][1])
    plt.show()
    plane+=1


Refine2DVertex works on CircleVertex's output.
Let's first take a look at the output of CircleVertex in these images.
Below we show:<br>
0) Global PCA per contour (shown in blue color lines) <br> 
1) Crossing of all PCAs in 0) (shown in blue points) <br>
2) Found rough vertex region in circle (shown in cyan) <br><br>


In [ ]:
dm = mgr.DataManager()

import matplotlib.patches as patches
xranges=((500,700),(500,700),(500,700))
yranges=((900,1000),(750,900),(1400,1500))

for plane in xrange(3):
    print 'plane',plane
    fig,ax=plt.subplots(figsize=(20,20))
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    ax.set_ylim(yranges[plane][0],yranges[plane][1])
    ax.set_xlim(xranges[plane][0],xranges[plane][1])
    ix=-1

    cv_data=dm.Data(2)
    
    cv_inter_pts_v=cv_data._ipoints_v_v_v[plane]
    cv_ctor_lines_v=cv_data._ctor_lines_v_v_v[plane]

    for cv_ctor_lines in cv_ctor_lines_v:
    
        ix+=1
        color=colormap[ix]
        for line in cv_ctor_lines:
            plt.plot([0,900],[line.y(0),line.y(900)],alpha=0.7,color=color)
        
        for inter_pt in cv_inter_pts_v[ix]:
            plt.plot(inter_pt.x,inter_pt.y,'o',markersize=10,color=color,alpha=0.9)   
        

    cv=larbysimg.Manager().GetClusterAlg(3)
    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    cv_data=dm.Data(3)
    circle_v = cv_data._circledata_v_v[plane]
    for circle in circle_v:
        c=patches.Circle((circle.center.x,circle.center.y),10,ec='cyan',alpha=0.5,fc='none',lw=10)
        ax.add_patch(c)
        
    ax=plt.gca()
    ax.set_aspect(1.0)
    plt.show()

Rough estimate on vertex region looks OK.
Next look at Refine2DVertex's search path for true vertex point.

In [ ]:
import matplotlib.patches as patches
xranges=((600,700),(575,675),(575,700))
yranges=((900,1000),(775,875),(1375,1475))
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(20,20))
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    #shape_img[:,::-1]=shape_img
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    #ax.set_ylim(np.min(nz_pixels[0])+50,np.max(nz_pixels[0])-40)
    #ax.set_xlim(np.min(nz_pixels[1])+250,np.max(nz_pixels[1])-90)
    #ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
    #ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    #ax.set_ylim(875,975)
    #ax.set_xlim(600,700)
    ax.set_ylim(yranges[plane][0],yranges[plane][1])
    ax.set_xlim(xranges[plane][0],xranges[plane][1])
    ix=-1


    for cv_ctor_lines in cv_ctor_lines_v:
    
        ix+=1
        color=colormap[ix]
        for line in cv_ctor_lines:
            pass#plt.plot([0,900],[line.y(0),line.y(900)],alpha=0.7,color=color)
        
        for inter_pt in cv_inter_pts_v[ix]:
            pass#plt.plot(inter_pt.x,inter_pt.y,'o',markersize=10,color=color,alpha=0.9)   
        

    cv=larbysimg.Manager().GetClusterAlg(3)
    mgr=larbysimg.Manager()
    dm=mgr.DataManager()

    ref_data = dm.Data(4)

    for idx in xrange(ref_data._xs_vv[plane].size()):
        pt = ref_data._xs_vv[plane][idx]
        plt.plot([pt.x],[pt.y],color='r',marker='o',markersize=20)
        print 'Crossing Pt.',idx,'...',pt.x,pt.y
        #print ref_data._x_vv[0][idx],ref_data._y_vv[0][idx]
    for l in ref_data._pca_vv[plane]:
        plt.plot([0,900],[l.y(0),l.y(900)],'-',color='orange',linewidth=2)
        #print l.pt.x,l.pt.y,l.dir.x,l.dir.y    

    print ref_data._circle_trav_vv[plane].size()
    for circle in ref_data._circle_trav_vv[plane]:
        c=patches.Circle((circle.center.x,circle.center.y),circle.radius,ec='cyan',alpha=0.2,fc='none',lw=10)
        ax.add_patch(c)
   
    # Show final circle
    for idx in xrange(ref_data._cand_score_vv.size()):
        score = ref_data._cand_score_vv[plane][idx]
        cand_vtx = ref_data._cand_vtx_vv[plane][idx]
        print 'Candidate vtx:',cand_vtx.x,cand_vtx.y,'... score:',score
        plt.plot([cand_vtx.x],[cand_vtx.y],marker='o',color='blue',markersize=15)

    ax=plt.gca()
    ax.set_aspect(1.0)
    plt.savefig
    plt.show()

What is shown in the above plots:

0) Red points denote the crossing point between the rough vertex's circle and particle trajectories. A continuous pixels that consist by one crossing are merged into one point by taking the x/y average. <br><br>
1) At each red point, using 5x5 square patch, local PCA is computed and shown by extending it to the image edge. <br><br>
2) For each red point with an extended PCA (orange line), we create a circle with user configurable radius. Then we move this circle along the orange line within the region defined by a user. Each time we move a circle, we compute again crossing point of particles' trajectory and a circumference, and compute local PCA using 5x5 square patch. We then compare the angle between such local PCA and a line that connects circle's center to a subject crossing point. The sum of this angle is used to minimize. <br><br>
3) After scanning along the orange line the algorithm suggests the point where angle diff of two lines are minimal = likely at the vertex. Overlapped cyan circles are from the steps taken by the algorithm <br><br>

Below is a python implementation made by myself for code development.

In [ ]:
import cv2

# pick ones closest in time
cx_v=[641.995788574,643.303588867,643.762451172 ]
cy_v=[936.084777832,823.297058105,1424.29443359 ]
#cx_v=[648.477050781,643.303588867,643.762451172 ]
#cy_v=[940.827514648,823.297058105,1424.29443359 ]
xranges=((600,700),(575,675),(575,700))
yranges=((900,1000),(775,875),(1375,1475))

#cx,cy = (641.995788574,936.084777832)
#cx,cy = (648.477050781,940.827514648)
#cx,cy = (642.,935.)
for plane in xrange(3):

    fig,ax = plt.subplots(figsize=(10,10))
    cx = cx_v[plane]
    cy = cy_v[plane]

    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    pimg=cv2.linearPolar(shape_img,(cx,cy),50,flags=cv2.WARP_FILL_OUTLIERS)
    col = 10./20. * pimg.shape[1]
    xv=[]
    yv=[]

    row_coll_v=[[]]
    combine_last = False

    for row in xrange(pimg.shape[0]):
        if pimg[row][col] < 1: 
            if len(row_coll_v[-1]): row_coll_v.append([])
            continue
        row_coll_v[-1].append(row)
    
    for idx in xrange(len(row_coll_v)):
        row_coll_v[idx] = np.array(row_coll_v[idx])

    # Check if the last element should be merged or not
    if len(row_coll_v) > 1:
        if row_coll_v[0][0] == 0 and row_coll_v[-1][-1] == pimg.shape[0]-1:
            row_coll_v[-1] -= (pimg.shape[0] - 1)
            row_coll_v[0] = np.concatenate((row_coll_v[0],row_coll_v[-1]),axis=0)
            row_coll_v.pop()
        for row_coll in row_coll_v:

            ar = np.array(row_coll)
            if len(row_coll) < 1: continue
            ave_row = ar.mean()
            if ave_row < 0:
                ave_row += (pimg.shape[0]-1)
            xv.append(cx + 10 * np.cos(2 * 3.1415 * float(ave_row/pimg.shape[0])))
            yv.append(cy + 10 * np.sin(2 * 3.1415 * float(ave_row/pimg.shape[0])))
            plt.plot([col],[ave_row],marker='o',color='red',markersize=10)
    
    plt.imshow(pimg,cmap='Greys',interpolation='none')

    fig,ax = plt.subplots(figsize=(20,20))
    for idx in xrange(len(xv)):
        plt.plot([xv[idx]],[yv[idx]],marker='o',color='yellow', markersize=10)
        pt1 = Vector(DTYPE)(cx,cy)
        pt2 = Vector(DTYPE)(xv[idx],yv[idx])
        d = geo2d.dir(pt1,pt2)
        l = geo2d.HalfLine(DTYPE)(pt1,d)
        if d.x < 0:
            plt.plot([cx,0],[cy,l.y(0.)],marker='o',color='red',linestyle='--',linewidth=2)
        else:
            plt.plot([cx,shape_img.shape[1]],[cy,l.y(shape_img.shape[1])],marker='o',color='red',linestyle='--',linewidth=2)
    
    c=patches.Circle((cx,cy),10,ec='cyan',fc='none',alpha=0.5,lw=10)
    ax.add_patch(c)

    plt.plot([cx],[cy],marker='o',color='red',markersize=10)
    
    nz_pixels=np.where(shape_img>0.0)
    ax.set_ylim(yranges[plane][0],yranges[plane][1])
    ax.set_xlim(xranges[plane][0],xranges[plane][1])
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
